In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
main_data_df = pd.read_csv('./prepped_data/main_data_baseline_checkpoint_3.csv')
print(main_data_df.shape)
main_data_df.head()

(4441256, 18)
Wall time: 6.48 s


,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,도로명주소,X좌표,Y좌표,주택가격지수
0,매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,79.100
1,매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,79.213
2,매매,77.75,29,55000,0,7,1988,707.395498,2014,1,10.915088,6.561590,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,79.451
3,매매,77.75,10,59500,0,4,1988,765.273312,2014,11,10.993732,6.640233,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,82.100
4,매매,77.75,28,75000,0,5,1988,964.630225,2016,3,11.225243,6.871745,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,89.466


In [3]:
basedir = './한국부동산원/전월세전환율/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.xlsx')]
filenames

['월세이율_2006.xlsx',
 '월세이율_2007.xlsx',
 '월세이율_2008.xlsx',
 '월세이율_2009.xlsx',
 '월세이율_2010.xlsx',
 '지역별_전월세전환율_아파트_시군구별.xlsx',
 '지역별_전월세전환율_연립다세대_시군구권역별.xlsx',
 '지역별_전월세전환율_종합_시군구권역별.xlsx']

In [4]:
main_data_df['전월세매매구분'].value_counts()

매매    1863695
전세    1743682
월세     833879
Name: 전월세매매구분, dtype: int64

In [5]:
monthly_charge_df = main_data_df[main_data_df['전월세매매구분'] == '월세']

In [6]:
monthly_charge_df['지번주소'].isna().sum()

229

In [7]:
monthly_charge_df.dropna(subset=['지번주소'], inplace=True)

In [8]:
monthly_charge_df['구'] = monthly_charge_df['지번주소'].apply(lambda x: x.split(' ')[1])

In [32]:
강북 = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구',
      '은평구', '서대문구', '마포구']
강남 = ['양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

In [38]:
monthly_charge_df['년'].value_counts()

2019    101653
2016    100214
2017     97550
2015     97300
2018     95272
2020     92999
2014     78413
2013     68691
2012     53741
2011     47817
Name: 년, dtype: int64

In [41]:
filenames

['월세이율_2006.xlsx',
 '월세이율_2007.xlsx',
 '월세이율_2008.xlsx',
 '월세이율_2009.xlsx',
 '월세이율_2010.xlsx',
 '지역별_전월세전환율_아파트_시군구별.xlsx',
 '지역별_전월세전환율_연립다세대_시군구권역별.xlsx',
 '지역별_전월세전환율_종합_시군구권역별.xlsx']

In [9]:
monthly_apart_df = monthly_charge_df[monthly_charge_df['건물종류'] == '아파트']

In [10]:
apart_rate_df = pd.read_excel(basedir + filenames[-3], header=10, index_col=[0,1])
print(apart_rate_df.shape)
apart_rate_df.head()

(26, 121)


주택유형  2011년 01월  2011년 02월  2011년 03월  2011년 04월  2011년 05월  \
지 역                                                                   
서울  NaN  아파트        7.4        7.4        7.4        7.4        7.4   
    종로구  아파트        7.6        7.6        7.6        7.6        7.6   
    중구   아파트        7.4        7.4        7.4        7.4        7.4   
    용산구  아파트        7.2        7.2        7.2        7.2        7.2   
    성동구  아파트        7.2        7.2        7.2        7.2        7.2   

         2011년 06월  2011년 07월  2011년 08월  2011년 09월  2011년 10월  2011년 11월  \
지 역                                                                         
서울  NaN        7.4        7.3        7.3        7.2        7.2        7.2   
    종로구        7.6        7.6        7.4        7.5        7.6        7.5   
    중구         7.4        7.3        7.2        7.0        7.1        6.9   
    용산구        7.2        7.2        7.4        7.3        7.4        7.4   
    성동구        7.2        7.2        7.2        7.2        7.3        7.2   

         2011년 12월  2012년 01월  2012년 02월  2012년 03월  2012년 04월  2012년 05월  \
지 역                                                                         
서울  NaN        7.1        7.1        7.1        7.0        7.0        7.0   
    종로구        7.2        6.8        6.8        6.9        6.8        6.9   
    중구         6.9        7.1        7.2        7.2        7.1        7.1   
    용산구        7.2        7.1        6.9        6.9        6.8        6.9   
    성동구        7.2        7.2        7.1        7.1        7.0        6.9   

         2012년 06월  2012년 07월  2012년 08월  2012년 09월  2012년 10월  2012년 11월  \
지 역                                                                         
서울  NaN        6.9        6.9        6.9        6.9        6.9        6.8   
    종로구        7.1        7.0        6.9        6.9        6.8        6.7   
    중구         7.1        6.9        6.9        6.9        6.9        6.9   
    용산구        6.9        6.9        7.0        7.0        6.9        6.8   
    성동구        7.0        6.9        7.0        6.9        6.9        6.9   

         2012년 12월  2013년 01월  2013년 02월  2013년 03월  2013년 04월  2013년 05월  \
지 역                                                                         
서울  NaN        6.7        6.7        6.7        6.6        6.6        6.6   
    종로구        6.9        7.0        6.8        6.9        6.9        6.7   
    중구         6.9        6.8        6.8        6.7        6.7        6.6   
    용산구        6.8        6.7        6.5        6.5        6.5        6.6   
    성동구        6.9        6.9        6.8        6.7        6.7        6.6   

         2013년 06월  2013년 07월  2013년 08월  2013년 09월  2013년 10월  2013년 11월  \
지 역                                                                         
서울  NaN        6.5        6.5        6.4        6.4        6.3        6.2   
    종로구        6.5        6.7        6.5        6.5        6.4        6.3   
    중구         6.6        6.5        6.4        6.3        6.1        6.0   
    용산구        6.4        6.5        6.5        6.4        6.3        6.2   
    성동구        6.5        6.5        6.3        6.3        6.2        6.1   

         2013년 12월  2014년 01월  2014년 02월  2014년 03월  2014년 04월  2014년 05월  \
지 역                                                                         
서울  NaN        6.1        6.0        6.0        5.9        5.9        5.8   
    종로구        6.2        6.0        6.3        6.2        6.1        6.4   
    중구         6.0        6.0        5.8        5.7        5.8        5.8   
    용산구        6.2        6.0        6.0        5.9        5.9        5.8   
    성동구        6.1        6.0        5.9        5.7        5.7        5.6   

         2014년 06월  2014년 07월  2014년 08월  2014년 09월  2014년 10월  2014년 11월  \
지 역                                                                         
서울  NaN        5.8        5.8        5.8        5.8        5.7        5.6   
    종로구        6.5        6.6        6.

In [11]:
apart_rate_df.drop(columns=['주택유형'], inplace=True)

In [12]:
apart_rate_df = apart_rate_df.loc['서울']
apart_rate_df.head()

,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
NaN,7.4,7.4,7.4,7.4,7.4,7.4,7.3,7.3,7.2,7.2,7.2,7.1,7.1,7.1,7.0,7.0,7.0,6.9,6.9,6.9,6.9,6.9,6.8,6.7,6.7,6.7,6.6,6.6,6.6,6.5,6.5,6.4,6.4,6.3,6.2,6.1,6.0,6.0,5.9,5.9,5.8,5.8,5.8,5.8,5.8,5.7,5.6,5.5,5.5,5.4,5.3,5.3,5.2,5.1,5.1,5.0,4.9,4.8,4.8,4.7,4.6,4.6,4.5,4.5,4.5,4.5,4.4,4.4,4.4,4.3,4.2,4.2,4.2,4.2,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.0,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.1,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.9,4.0
종로구,7.6,7.6,7.6,7.6,7.6,7.6,7.6,7.4,7.5,7.6,7.5,7.2,6.8,6.8,6.9,6.8,6.9,7.1,7.0,6.9,6.9,6.8,6.7,6.9,7.0,6.8,6.9,6.9,6.7,6.5,6.7,6.5,6.5,6.4,6.3,6.2,6.0,6.3,6.2,6.1,6.4,6.5,6.6,6.2,6.3,6.0,6.0,6.3,6.0,6.0,6.0,6.0,6.0,6.0,6.1,6.0,6.0,5.6,5.8,5.4,5.5,5.2,5.2,5.4,5.4,5.5,5.4,5.3,5.3,5.1,5.2,5.2,5.2,5.3,5.0,4.7,4.4,4.3,4.2,4.0,4.0,4.1,4.0,4.1,4.1,4.4,4.5,4.5,4.5,4.8,4.6,4.5,4.6,4.4,4.3,4.2,4.2,4.2,4.2,4.1,4.1,4.2,4.2,4.2,4.2,4.2,4.2,4.0,4.1,4.2,4.2,4.1,4.1,4.1,4.1,4.5,4.2,4.1,4.1,4.1
중구,7.4,7.4,7.4,7.4,7.4,7.4,7.3,7.2,7.0,7.1,6.9,6.9,7.1,7.2,7.2,7.1,7.1,7.1,6.9,6.9,6.9,6.9,6.9,6.9,6.8,6.8,6.7,6.7,6.6,6.6,6.5,6.4,6.3,6.1,6.0,6.0,6.0,5.8,5.7,5.8,5.8,5.7,5.8,5.7,5.6,5.6,5.5,5.5,5.5,5.5,5.4,5.3,5.2,5.1,5.1,5.0,5.0,5.0,4.8,4.7,4.6,4.6,4.6,4.6,4.7,4.7,4.6,4.6,4.5,4.5,4.3,4.2,4.2,4.2,4.2,4.2,4.2,4.3,4.2,4.1,4.1,4.1,4.1,4.0,4.0,4.1,4.1,4.1,4.1,4.1,4.2,4.3,4.2,4.2,4.2,4.2,4.1,4.1,4.1,4.1,4.1,4.1,4.2,4.1,4.0,4.1,4.0,4.0,4.1,4.0,4.1,4.0,4.1,4.1,4.0,4.1,4.1,4.1,4.0,4.1
용산구,7.2,7.2,7.2,7.2,7.2,7.2,7.2,7.4,7.3,7.4,7.4,7.2,7.1,6.9,6.9,6.8,6.9,6.9,6.9,7.0,7.0,6.9,6.8,6.8,6.7,6.5,6.5,6.5,6.6,6.4,6.5,6.5,6.4,6.3,6.2,6.2,6.0,6.0,5.9,5.9,5.8,5.7,5.6,5.6,5.7,5.6,5.6,5.6,5.6,5.5,5.4,5.4,5.3,5.3,5.2,5.1,5.1,5.0,4.9,5.0,5.0,4.8,4.7,4.6,4.6,4.5,4.6,4.5,4.5,4.4,4.4,4.3,4.3,4.3,4.2,4.3,4.3,4.3,4.4,4.4,4.3,4.3,4.3,4.2,4.1,4.2,4.2,4.2,4.2,4.2,4.2,4.2,4.2,4.2,4.2,4.3,4.3,4.3,4.2,4.2,4.3,4.3,4.3,4.3,4.3,4.2,4.2,4.1,4.2,4.2,4.1,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.1,4.2
성동구,7.2,7.2,7.2,7.2,7.2,7.2,7.2,7.2,7.2,7.3,7.2,7.2,7.2,7.1,7.1,7.0,6.9,7.0,6.9,7.0,6.9,6.9,6.9,6.9,6.9,6.8,6.7,6.7,6.6,6.5,6.5,6.3,6.3,6.2,6.1,6.1,6.0,5.9,5.7,5.7,5.6,5.6,5.5,5.5,5.5,5.5,5.5,5.4,5.4,5.2,5.1,5.1,5.1,5.0,5.0,4.9,4.9,4.7,4.7,4.6,4.5,4.5,4.4,4.3,4.3,4.3,4.3,4.3,4.3,4.2,4.2,4.2,4.2,4.2,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.0,4.0,4.0,4.1,4.0,4.1,4.1,4.0,4.0,4.1,4.1,4.1,4.1,4.1,4.1,4.0,4.0,4.1,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.9,3.9,3.9,4.0


In [13]:
idx_names = []
for i, idx in enumerate(apart_rate_df.index):
    if i == 0:
        idx_names.append('서울전체')
    else:
        idx_names.append(idx)
idx_names

['서울전체',
 '종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

In [14]:
apart_rate_df.index = idx_names

In [15]:
apart_rate_df.head()

,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
서울전체,7.4,7.4,7.4,7.4,7.4,7.4,7.3,7.3,7.2,7.2,7.2,7.1,7.1,7.1,7.0,7.0,7.0,6.9,6.9,6.9,6.9,6.9,6.8,6.7,6.7,6.7,6.6,6.6,6.6,6.5,6.5,6.4,6.4,6.3,6.2,6.1,6.0,6.0,5.9,5.9,5.8,5.8,5.8,5.8,5.8,5.7,5.6,5.5,5.5,5.4,5.3,5.3,5.2,5.1,5.1,5.0,4.9,4.8,4.8,4.7,4.6,4.6,4.5,4.5,4.5,4.5,4.4,4.4,4.4,4.3,4.2,4.2,4.2,4.2,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.0,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.1,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.9,4.0
종로구,7.6,7.6,7.6,7.6,7.6,7.6,7.6,7.4,7.5,7.6,7.5,7.2,6.8,6.8,6.9,6.8,6.9,7.1,7.0,6.9,6.9,6.8,6.7,6.9,7.0,6.8,6.9,6.9,6.7,6.5,6.7,6.5,6.5,6.4,6.3,6.2,6.0,6.3,6.2,6.1,6.4,6.5,6.6,6.2,6.3,6.0,6.0,6.3,6.0,6.0,6.0,6.0,6.0,6.0,6.1,6.0,6.0,5.6,5.8,5.4,5.5,5.2,5.2,5.4,5.4,5.5,5.4,5.3,5.3,5.1,5.2,5.2,5.2,5.3,5.0,4.7,4.4,4.3,4.2,4.0,4.0,4.1,4.0,4.1,4.1,4.4,4.5,4.5,4.5,4.8,4.6,4.5,4.6,4.4,4.3,4.2,4.2,4.2,4.2,4.1,4.1,4.2,4.2,4.2,4.2,4.2,4.2,4.0,4.1,4.2,4.2,4.1,4.1,4.1,4.1,4.5,4.2,4.1,4.1,4.1
중구,7.4,7.4,7.4,7.4,7.4,7.4,7.3,7.2,7.0,7.1,6.9,6.9,7.1,7.2,7.2,7.1,7.1,7.1,6.9,6.9,6.9,6.9,6.9,6.9,6.8,6.8,6.7,6.7,6.6,6.6,6.5,6.4,6.3,6.1,6.0,6.0,6.0,5.8,5.7,5.8,5.8,5.7,5.8,5.7,5.6,5.6,5.5,5.5,5.5,5.5,5.4,5.3,5.2,5.1,5.1,5.0,5.0,5.0,4.8,4.7,4.6,4.6,4.6,4.6,4.7,4.7,4.6,4.6,4.5,4.5,4.3,4.2,4.2,4.2,4.2,4.2,4.2,4.3,4.2,4.1,4.1,4.1,4.1,4.0,4.0,4.1,4.1,4.1,4.1,4.1,4.2,4.3,4.2,4.2,4.2,4.2,4.1,4.1,4.1,4.1,4.1,4.1,4.2,4.1,4.0,4.1,4.0,4.0,4.1,4.0,4.1,4.0,4.1,4.1,4.0,4.1,4.1,4.1,4.0,4.1
용산구,7.2,7.2,7.2,7.2,7.2,7.2,7.2,7.4,7.3,7.4,7.4,7.2,7.1,6.9,6.9,6.8,6.9,6.9,6.9,7.0,7.0,6.9,6.8,6.8,6.7,6.5,6.5,6.5,6.6,6.4,6.5,6.5,6.4,6.3,6.2,6.2,6.0,6.0,5.9,5.9,5.8,5.7,5.6,5.6,5.7,5.6,5.6,5.6,5.6,5.5,5.4,5.4,5.3,5.3,5.2,5.1,5.1,5.0,4.9,5.0,5.0,4.8,4.7,4.6,4.6,4.5,4.6,4.5,4.5,4.4,4.4,4.3,4.3,4.3,4.2,4.3,4.3,4.3,4.4,4.4,4.3,4.3,4.3,4.2,4.1,4.2,4.2,4.2,4.2,4.2,4.2,4.2,4.2,4.2,4.2,4.3,4.3,4.3,4.2,4.2,4.3,4.3,4.3,4.3,4.3,4.2,4.2,4.1,4.2,4.2,4.1,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.1,4.2
성동구,7.2,7.2,7.2,7.2,7.2,7.2,7.2,7.2,7.2,7.3,7.2,7.2,7.2,7.1,7.1,7.0,6.9,7.0,6.9,7.0,6.9,6.9,6.9,6.9,6.9,6.8,6.7,6.7,6.6,6.5,6.5,6.3,6.3,6.2,6.1,6.1,6.0,5.9,5.7,5.7,5.6,5.6,5.5,5.5,5.5,5.5,5.5,5.4,5.4,5.2,5.1,5.1,5.1,5.0,5.0,4.9,4.9,4.7,4.7,4.6,4.5,4.5,4.4,4.3,4.3,4.3,4.3,4.3,4.3,4.2,4.2,4.2,4.2,4.2,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.1,4.0,4.0,4.0,4.0,4.0,4.0,4.1,4.0,4.1,4.1,4.0,4.0,4.1,4.1,4.1,4.1,4.1,4.1,4.0,4.0,4.1,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.9,3.9,3.9,4.0


In [16]:
main_data_df['전환된전세'] = np.nan

In [17]:
for col in tqdm(apart_rate_df.columns):
    splitted = col.split(' ')
    year = int(splitted[0].replace('년', ''))
    month = int(splitted[1].replace('월', ''))
    
    yearmonth_df = monthly_apart_df[(monthly_apart_df['년'] == year)&(monthly_apart_df['월'] == month)]
    
    for gu in yearmonth_df['구'].unique():
        gu_df = yearmonth_df[yearmonth_df['구'] == gu]
        
        main_data_df.loc[gu_df.index, '전환된전세'] = gu_df['금액(만원)'] + ((gu_df['월세(만원)'] * 12) / (apart_rate_df.loc[gu, col] / 100))

100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:56<00:00,  2.12it/s]


In [18]:
filenames

['월세이율_2006.xlsx',
 '월세이율_2007.xlsx',
 '월세이율_2008.xlsx',
 '월세이율_2009.xlsx',
 '월세이율_2010.xlsx',
 '지역별_전월세전환율_아파트_시군구별.xlsx',
 '지역별_전월세전환율_연립다세대_시군구권역별.xlsx',
 '지역별_전월세전환율_종합_시군구권역별.xlsx']

In [19]:
%%time
yunrip_rate_df = pd.read_excel(basedir + filenames[-2], header=10, index_col=[0,1])
yunrip_rate_df.drop(columns=['주택유형'], inplace=True)
yunrip_rate_df = yunrip_rate_df.loc['서울']
idx_names = []
for i, v in enumerate(yunrip_rate_df.index):
    if i == 0:
        idx_names.append('서울전체')
    else:
        idx_names.append(v)
yunrip_rate_df.index = idx_names
print(yunrip_rate_df.shape)
yunrip_rate_df

(8, 120)
Wall time: 24 ms


,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
서울전체,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.3,8.3,8.2,8.2,8.3,8.3,8.3,8.2,8.2,8.2,8.2,8.1,8.0,8.0,7.9,7.9,7.9,7.8,7.8,7.8,7.8,7.8,7.7,7.7,7.6,7.6,7.5,7.4,7.5,7.4,7.4,7.3,7.2,7.2,7.1,7.1,6.9,6.8,6.7,6.6,6.6,6.6,6.7,6.7,6.6,6.5,6.4,6.3,6.2,6.0,5.9,5.9,5.8,5.8,5.8,5.8,5.7,5.7,5.6,5.6,5.5,5.4,5.4,5.3,5.3,5.2,5.2,5.2,5.1,5.1,5.1,5.0,5.0,4.9,4.8,4.8,4.7,4.8,4.8,4.8,4.8,4.8,4.8,4.8,4.7,4.6,4.6,4.5,4.5,4.5,4.6,4.6,4.7,4.6,4.6,4.5,4.5,4.4,4.3,4.3,4.2,4.3,4.2,4.3,4.3,4.2,4.2,4.2,4.1,4.0,4.0,4.0,3.9
강북지역,8.4,8.4,8.4,8.4,8.4,8.4,8.5,8.4,8.4,8.3,8.3,8.4,8.4,8.4,8.4,8.4,8.4,8.3,8.3,8.2,8.2,8.1,8.1,8.0,8.0,8.0,8.0,8.0,8.0,8.0,7.9,7.9,7.8,7.8,7.7,7.7,7.7,7.8,7.7,7.6,7.5,7.6,7.5,7.2,7.2,7.1,6.9,6.9,6.9,7.2,7.1,7.1,7.1,7.0,6.8,6.7,6.5,6.4,6.4,6.2,6.2,6.3,6.3,6.2,6.2,6.2,6.0,6.0,5.9,5.9,5.8,5.8,5.7,5.7,5.7,5.7,5.6,5.6,5.5,5.4,5.3,5.2,5.1,5.1,5.2,5.2,5.3,5.3,5.3,5.2,5.2,5.0,5.0,4.8,4.9,4.9,4.9,5.0,5.0,5.0,4.9,4.9,4.9,4.8,4.7,4.6,4.6,4.6,4.5,4.5,4.6,4.6,4.5,4.5,4.4,4.3,4.2,4.2,4.2,4.1
강남지역,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.3,8.3,8.2,8.1,8.2,8.2,8.2,8.1,8.1,8.0,8.0,8.0,7.8,7.9,7.8,7.8,7.8,7.8,7.7,7.6,7.6,7.6,7.6,7.5,7.4,7.4,7.4,7.3,7.3,7.2,7.1,7.1,7.0,7.0,6.9,6.9,6.7,6.6,6.5,6.4,6.4,6.4,6.4,6.4,6.3,6.2,6.1,6.0,6.0,5.8,5.7,5.7,5.6,5.5,5.5,5.5,5.4,5.4,5.3,5.3,5.2,5.1,5.1,5.0,5.0,5.0,4.9,4.9,4.8,4.8,4.8,4.8,4.7,4.7,4.6,4.6,4.5,4.5,4.6,4.6,4.6,4.6,4.6,4.6,4.5,4.5,4.4,4.4,4.3,4.3,4.4,4.4,4.5,4.4,4.4,4.3,4.3,4.3,4.2,4.2,4.0,4.1,4.1,4.1,4.1,4.1,4.1,4.0,4.0,3.9,3.9,3.9,3.8
도심권,9.2,9.2,9.2,9.2,9.2,9.2,9.7,9.1,9.3,9.0,9.2,9.2,8.9,9.6,9.2,9.1,9.0,8.7,8.4,8.4,8.3,8.4,8.5,7.9,8.4,8.1,8.3,8.2,8.2,8.2,8.2,8.3,8.1,8.0,7.9,7.9,8.0,8.3,8.1,8.2,8.3,8.3,7.9,7.3,7.2,7.2,7.0,7.0,7.0,7.3,7.1,6.9,7.2,6.9,6.8,6.7,6.6,6.3,6.3,6.4,6.1,6.3,6.2,6.3,6.3,6.2,6.1,6.3,6.0,6.0,5.7,5.8,5.7,5.7,5.8,6.0,5.9,5.9,5.8,5.6,5.6,5.4,5.4,5.3,5.5,5.6,5.7,5.7,5.6,5.4,5.5,5.3,5.3,5.1,5.2,5.2,5.2,5.2,5.3,5.3,5.3,5.3,5.2,5.1,5.1,5.0,5.0,4.8,4.8,4.9,4.9,5.0,5.1,5.1,5.0,4.9,4.8,4.6,4.5,4.5
동북권,8.2,8.2,8.2,8.2,8.2,8.2,8.3,8.2,8.2,8.1,8.0,8.1,8.2,8.1,8.2,8.3,8.4,8.2,8.2,8.0,8.0,7.8,7.9,8.0,7.9,8.0,8.0,8.0,8.0,8.0,7.9,7.8,7.8,7.7,7.7,7.6,7.6,7.7,7.6,7.4,7.3,7.4,7.3,7.0,7.0,7.0,6.8,6.8,6.8,7.2,7.2,7.2,7.2,7.0,6.8,6.5,6.3,6.3,6.2,6.1,6.1,6.3,6.3,6.2,6.3,6.2,6.1,5.9,5.8,5.8,5.7,5.7,5.7,5.8,5.7,5.6,5.6,5.6,5.5,5.4,5.2,5.1,5.0,5.0,5.0,5.1,5.2,5.2,5.1,5.1,5.0,4.8,4.8,4.7,4.7,4.7,4.7,4.8,4.8,4.9,4.8,4.7,4.7,4.6,4.5,4.3,4.3,4.4,4.3,4.4,4.5,4.5,4.3,4.3,4.2,4.1,4.0,3.9,3.9,3.9
서북권,8.4,8.4,8.4,8.4,8.4,8.4,8.5,8.3,8.3,8.3,8.4,8.5,8.4,8.5,8.5,8.4,8.4,8.4,8.3,8.5,8.5,8.4,8.2,8.1,8.1,8.0,7.9,8.0,8.0,7.9,7.9,7.9,7.8,7.7,7.7,7.7,7.7,7.8,7.6,7.5,7.5,7.5,7.5,7.3,7.4,7.3,7.1,6.9,7.0,7.2,7.1,7.1,7.0,7.1,6.9,6.8,6.6,6.5,6.5,6.3,6.2,6.3,6.3,6.2,6.2,6.2,6.0,6.0,5.9,5.9,5.9,5.8,5.7,5.7,5.7,5.6,5.6,5.5,5.4,5.3,5.3,5.3,5.1,5.1,5.2,5.2,5.2,5.2,5.3,5.3,5.2,5.1,5.1,

In [20]:
monthly_yunrip_df = monthly_charge_df[monthly_charge_df['건물종류'] == '다세대연립']
print(monthly_yunrip_df.shape)
monthly_yunrip_df.head()

(262319, 19)


,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,도로명주소,X좌표,Y좌표,주택가격지수,구
168507,월세,84.99,11,10000,160,7,2005,117.660901,2011,11,9.210340,4.767807,다세대연립,서울특별시 강남구 역삼동 831-45,서울특별시 강남구 역삼로 109-0,958531.358233,1.943929e+06,NaN,강남구
181074,월세,41.15,1,10000,40,6,1997,243.013366,2020,7,9.210340,5.493116,다세대연립,서울특별시 강남구 역삼동 835-33,서울특별시 강남구 도곡로11길 10-0,958953.370336,1.943695e+06,NaN,강남구
182654,월세,29.88,8,3000,100,2,2015,100.401606,2020,6,8.006368,4.609178,다세대연립,서울특별시 강남구 역삼동 834-36,서울특별시 강남구 역삼로10길 13-0,958827.041299,1.943891e+06,NaN,강남구
182655,월세,18.97,30,1000,70,4,2015,52.714813,2020,10,6.907755,3.964896,다세대연립,서울특별시 강남구 역삼동 834-36,서울특별시 강남구 역삼로10길 13-0,958827.041299,1.943891e+06,NaN,강남구
206152,월세,164.83,18,10000,250,1,1983,60.668568,2011,6,9.210340,4.105426,다세대연립,서울특별시 강남구 청담동 67-0,서울특별시 강남구 학동로97길 51-0,960603.498246,1.947173e+06,NaN,강남구


In [21]:
도심권 = ['종로구', '중구', '용산구']
동북권 = ['성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구']
서북권 = ['은평구', '서대문구', '마포구']
서남권 = ['양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구']
동남권 = ['서초구', '강남구', '송파구', '강동구']

In [22]:
for col in tqdm(yunrip_rate_df.columns):
    year = int(col.split(' ')[0].replace('년', ''))
    month = int(col.split(' ')[1].replace('월', ''))
    
    yearmonth_df = monthly_yunrip_df[(monthly_yunrip_df['년'] == year)&(monthly_yunrip_df['월'] == month)]
    
    for gu in yearmonth_df['구'].unique():
        gu_df = yearmonth_df[yearmonth_df['구'] == gu]
        
        if gu in 도심권:
            area = '도심권'
        elif gu in 동북권:
            area = '동북권'
        elif gu in 서북권:
            area = '서북권'
        elif gu in 서남권:
            area = '서남권'
        elif gu in 동남권:
            area = '동남권'
        else:
            raise ValueError('gu not in any of the areas')
        
        main_data_df.loc[gu_df.index, '전환된전세'] = gu_df['금액(만원)'] + ((gu_df['월세(만원)'] * 12) / (yunrip_rate_df.loc[area, col] / 100))

100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:55<00:00,  2.15it/s]


In [23]:
filenames

['월세이율_2006.xlsx',
 '월세이율_2007.xlsx',
 '월세이율_2008.xlsx',
 '월세이율_2009.xlsx',
 '월세이율_2010.xlsx',
 '지역별_전월세전환율_아파트_시군구별.xlsx',
 '지역별_전월세전환율_연립다세대_시군구권역별.xlsx',
 '지역별_전월세전환율_종합_시군구권역별.xlsx']

In [24]:
filenames[-1]

'지역별_전월세전환율_종합_시군구권역별.xlsx'

In [25]:
%%time
overall_rate_df = pd.read_excel(basedir + filenames[-1], header=10, index_col=[0,1])
overall_rate_df.drop(columns=['주택유형'], inplace=True)
overall_rate_df = overall_rate_df.loc['서울']
idx_names = []
for i, v in enumerate(overall_rate_df.index):
    if i == 0:
        idx_names.append('서울전체')
    else:
        idx_names.append(v)
overall_rate_df.index = idx_names
print(overall_rate_df.shape)
overall_rate_df

(8, 120)
Wall time: 25 ms


,2011년 01월,2011년 02월,2011년 03월,2011년 04월,2011년 05월,2011년 06월,2011년 07월,2011년 08월,2011년 09월,2011년 10월,2011년 11월,2011년 12월,2012년 01월,2012년 02월,2012년 03월,2012년 04월,2012년 05월,2012년 06월,2012년 07월,2012년 08월,2012년 09월,2012년 10월,2012년 11월,2012년 12월,2013년 01월,2013년 02월,2013년 03월,2013년 04월,2013년 05월,2013년 06월,2013년 07월,2013년 08월,2013년 09월,2013년 10월,2013년 11월,2013년 12월,2014년 01월,2014년 02월,2014년 03월,2014년 04월,2014년 05월,2014년 06월,2014년 07월,2014년 08월,2014년 09월,2014년 10월,2014년 11월,2014년 12월,2015년 01월,2015년 02월,2015년 03월,2015년 04월,2015년 05월,2015년 06월,2015년 07월,2015년 08월,2015년 09월,2015년 10월,2015년 11월,2015년 12월,2016년 01월,2016년 02월,2016년 03월,2016년 04월,2016년 05월,2016년 06월,2016년 07월,2016년 08월,2016년 09월,2016년 10월,2016년 11월,2016년 12월,2017년 01월,2017년 02월,2017년 03월,2017년 04월,2017년 05월,2017년 06월,2017년 07월,2017년 08월,2017년 09월,2017년 10월,2017년 11월,2017년 12월,2018년 01월,2018년 02월,2018년 03월,2018년 04월,2018년 05월,2018년 06월,2018년 07월,2018년 08월,2018년 09월,2018년 10월,2018년 11월,2018년 12월,2019년 01월,2019년 02월,2019년 03월,2019년 04월,2019년 05월,2019년 06월,2019년 07월,2019년 08월,2019년 09월,2019년 10월,2019년 11월,2019년 12월,2020년 01월,2020년 02월,2020년 03월,2020년 04월,2020년 05월,2020년 06월,2020년 07월,2020년 08월,2020년 09월,2020년 10월,2020년 11월,2020년 12월
서울전체,8.6,8.6,8.6,8.6,8.6,8.6,8.5,8.4,8.3,8.2,8.2,8.2,8.2,8.2,8.2,8.1,8.1,8.1,8.0,8.0,7.9,7.9,7.8,7.8,7.8,7.9,7.8,7.8,7.8,7.8,7.7,7.6,7.5,7.4,7.3,7.3,7.3,7.3,7.3,7.2,7.2,7.2,7.2,7.1,7.1,7.0,6.9,6.9,6.8,6.7,6.7,6.6,6.6,6.5,6.5,6.4,6.4,6.2,6.2,6.1,6.0,6.0,6.0,6.0,6.0,6.0,6.0,5.9,5.8,5.7,5.7,5.6,5.6,5.6,5.6,5.6,5.6,5.6,5.5,5.5,5.4,5.4,5.3,5.3,5.3,5.3,5.4,5.4,5.4,5.4,5.4,5.4,5.4,5.3,5.3,5.2,5.2,5.2,5.2,5.3,5.3,5.3,5.3,5.2,5.2,5.1,5.1,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.9,4.9,4.9,4.8,4.7
강북지역,8.8,8.8,8.8,8.8,8.8,8.8,8.8,8.6,8.6,8.5,8.4,8.4,8.4,8.4,8.4,8.4,8.3,8.3,8.2,8.1,8.1,8.0,8.0,8.0,8.0,8.1,8.1,8.0,8.0,8.0,7.9,7.8,7.6,7.6,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.4,7.4,7.3,7.2,7.2,7.1,7.1,7.0,7.1,7.0,7.0,6.9,6.9,6.8,6.7,6.6,6.5,6.4,6.3,6.3,6.4,6.3,6.3,6.3,6.3,6.2,6.2,6.1,6.0,5.9,5.9,5.9,5.9,5.9,5.9,6.0,5.9,5.9,5.8,5.7,5.6,5.6,5.5,5.6,5.6,5.7,5.7,5.7,5.7,5.7,5.6,5.6,5.5,5.5,5.5,5.5,5.5,5.6,5.6,5.5,5.5,5.5,5.4,5.4,5.3,5.3,5.2,5.2,5.2,5.2,5.2,5.3,5.3,5.3,5.2,5.1,5.1,5.0,5.0
강남지역,8.3,8.3,8.3,8.3,8.3,8.3,8.3,8.2,8.1,8.0,8.0,8.0,8.0,8.0,8.0,7.9,7.9,7.9,7.9,7.8,7.8,7.8,7.7,7.7,7.7,7.6,7.6,7.6,7.6,7.5,7.5,7.4,7.3,7.3,7.2,7.2,7.1,7.0,7.0,7.0,7.0,7.0,7.0,7.0,6.9,6.9,6.8,6.7,6.6,6.5,6.5,6.4,6.3,6.3,6.2,6.2,6.2,6.1,6.0,5.8,5.8,5.7,5.7,5.7,5.7,5.7,5.7,5.7,5.6,5.5,5.5,5.4,5.4,5.3,5.3,5.3,5.3,5.3,5.3,5.3,5.2,5.2,5.2,5.1,5.1,5.1,5.1,5.2,5.2,5.2,5.2,5.2,5.2,5.1,5.1,5.0,5.0,5.0,5.0,5.0,5.1,5.1,5.1,5.0,5.0,4.9,4.9,4.8,4.8,4.7,4.7,4.7,4.7,4.8,4.7,4.7,4.7,4.7,4.6,4.5
도심권,9.2,9.2,9.2,9.2,9.2,9.2,9.2,9.0,8.9,9.0,9.0,8.7,8.6,8.7,8.5,8.5,8.4,8.3,8.2,8.3,8.2,8.3,8.3,8.2,8.3,8.3,8.4,8.2,8.3,8.2,8.2,8.1,7.8,7.7,7.7,7.7,7.7,7.8,7.7,7.8,7.8,7.8,7.7,7.5,7.5,7.5,7.5,7.5,7.4,7.4,7.4,7.2,7.2,7.2,7.0,6.9,6.8,6.6,6.5,6.4,6.4,6.4,6.4,6.5,6.4,6.4,6.4,6.4,6.4,6.2,6.1,6.1,6.2,6.1,6.0,6.1,6.0,6.0,6.0,5.8,5.8,5.8,5.8,5.7,5.7,5.9,5.9,5.9,6.0,6.0,6.1,5.8,5.7,5.6,5.4,5.3,5.4,5.6,5.6,5.6,5.7,5.7,5.7,5.6,5.6,5.5,5.5,5.4,5.2,5.3,5.3,5.3,5.4,5.4,5.3,5.3,5.2,5.1,5.0,4.9
동북권,8.7,8.7,8.7,8.7,8.7,8.7,8.7,8.4,8.4,8.3,8.2,8.2,8.2,8.2,8.2,8.2,8.2,8.1,8.1,8.0,7.9,7.8,7.8,7.8,7.8,8.0,8.0,7.9,7.9,7.9,7.8,7.7,7.6,7.5,7.4,7.3,7.4,7.4,7.4,7.3,7.3,7.3,7.3,7.2,7.1,7.0,7.0,6.9,6.9,6.9,6.9,6.9,6.8,6.8,6.7,6.6,6.5,6.4,6.3,6.3,6.3,6.3,6.3,6.3,6.2,6.2,6.2,6.1,6.0,5.9,5.9,5.8,5.9,5.9,5.9,5.9,6.0,5.9,5.9,5.8,5.7,5.6,5.6,5.5,5.6,5.6,5.6,5.6,5.7,5.7,5.6,5.5,5.5,5.5,5.4,5.5,5.5,5.5,5.5,5.6,5.5,5.5,5.5,5.4,5.3,5.3,5.2,5.2,5.2,5.2,5.2,5.2,5.2,5.3,5.3,5.2,5.1,5.1,5.0,4.9
서북권,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,8.9,8.8,8.7,8.7,8.7,8.7,8.7,8.7,8.7,8.7,8.6,8.5,8.5,8.4,8.3,8.3,8.2,8.2,8.2,8.2,8.2,8.1,8.0,7.9,7.8,7.7,7.6,7.7,7.7,7.8,7.7,7.7,7.7,7.6,7.6,7.5,7.5,7.4,7.3,7.3,7.2,7.3,7.2,7.1,7.1,7.1,6.9,6.9,6.8,6.7,6.6,6.5,6.4,6.5,6.4,6.4,6.4,6.4,6.3,6.3,6.2,6.2,6.0,6.0,5.8,5.9,5.9,5.8,5.9,5.8,5.8,5.7,5.6,5.5,5.5,5.5,5.6,5.6,5.7,5.6,5.7,5.8,5.7,5.7,5.6,

In [26]:
monthly_charge_df['건물종류'].value_counts()

아파트      426326
다세대연립    262319
오피스텔     145005
Name: 건물종류, dtype: int64

In [27]:
monthly_officetel_df = monthly_charge_df[monthly_charge_df['건물종류'] == '오피스텔']
print(monthly_officetel_df.shape)
monthly_officetel_df.head()

(145005, 19)


,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,도로명주소,X좌표,Y좌표,주택가격지수,구
27743,월세,55.21,5,10000,100,12,2007,181.126607,2011,1,9.210340,5.199196,오피스텔,서울특별시 강남구 논현동 241-1,서울특별시 강남구 학동로 338-0,959359.054275,1.946475e+06,NaN,강남구
27744,월세,72.93,18,20000,100,19,2007,274.235568,2011,5,9.903488,5.613987,오피스텔,서울특별시 강남구 논현동 241-1,서울특별시 강남구 학동로 338-0,959359.054275,1.946475e+06,NaN,강남구
27745,월세,85.76,25,10000,200,10,2007,116.604478,2014,8,9.210340,4.758788,오피스텔,서울특별시 강남구 논현동 241-1,서울특별시 강남구 학동로 338-0,959359.054275,1.946475e+06,NaN,강남구
27746,월세,75.94,17,5000,200,9,2007,65.841454,2014,10,8.517193,4.187250,오피스텔,서울특별시 강남구 논현동 241-1,서울특별시 강남구 학동로 338-0,959359.054275,1.946475e+06,NaN,강남구
27747,월세,75.94,23,30000,120,5,2007,395.048723,2015,11,10.308953,5.979009,오피스텔,서울특별시 강남구 논현동 241-1,서울특별시 강남구 학동로 338-0,959359.054275,1.946475e+06,NaN,강남구


In [28]:
for col in tqdm(overall_rate_df.columns):
    year = int(col.split(' ')[0].replace('년', ''))
    month = int(col.split(' ')[1].replace('월', ''))
    
    yearmonth_df = monthly_officetel_df[(monthly_officetel_df['년'] == year)&(monthly_officetel_df['월'] == month)]
    
    for gu in yearmonth_df['구'].unique():
        gu_df = yearmonth_df[yearmonth_df['구'] == gu]
        
        if gu in 도심권:
            area = '도심권'
        elif gu in 동북권:
            area = '동북권'
        elif gu in 서북권:
            area = '서북권'
        elif gu in 서남권:
            area = '서남권'
        elif gu in 동남권:
            area = '동남권'
        else:
            raise ValueError('gu not in any of the areas')
        
        main_data_df.loc[gu_df.index, '전환된전세'] = gu_df['금액(만원)'] + ((gu_df['월세(만원)'] * 12) / (overall_rate_df.loc[area, col] / 100))

100%|████████████████████████████████████████████████████████████████████████████████| 120/120 [00:57<00:00,  2.08it/s]


In [103]:
main_data_df.shape

(4441256, 18)

In [104]:
main_data_df['전환된전세'].isna().sum()

3607606

In [32]:
monthly_charge_df = main_data_df[main_data_df['전월세매매구분'] == '월세']
monthly_charge_df.shape

(833879, 19)

In [33]:
monthly_charge_df['전환된전세'].isna().sum()

229

In [34]:
nan_df = monthly_charge_df[monthly_charge_df['전환된전세'].isna()]
nan_df.shape

(229, 19)

In [35]:
main_data_df.drop(nan_df.index, inplace=True)

In [36]:
main_data_df.shape

(4441027, 19)

In [37]:
monthly_charge_df.drop(nan_df.index, inplace=True)

In [38]:
main_data_df.loc[monthly_charge_df.index, '금액(만원)'] = monthly_charge_df['전환된전세']

In [40]:
main_data_df['전월세매매구분'] = main_data_df['전월세매매구분'].apply(lambda x: '전세' if x == '월세' else x)

In [41]:
main_data_df['전월세매매구분'].value_counts()

전세    2577332
매매    1863695
Name: 전월세매매구분, dtype: int64

In [42]:
main_data_df.drop(columns=['월세(만원)'], inplace=True)

In [43]:
main_data_df.head()

,전월세매매구분,전용면적(㎡),계약일,금액(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,도로명주소,X좌표,Y좌표,주택가격지수,전환된전세
0,매매,77.75,8,57000.0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,79.100,NaN
1,매매,77.75,16,57000.0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,79.213,NaN
2,매매,77.75,29,55000.0,7,1988,707.395498,2014,1,10.915088,6.561590,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,79.451,NaN
3,매매,77.75,10,59500.0,4,1988,765.273312,2014,11,10.993732,6.640233,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,82.100,NaN
4,매매,77.75,28,75000.0,5,1988,964.630225,2016,3,11.225243,6.871745,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,89.466,NaN


In [44]:
main_data_df.drop(columns=['전환된전세'], inplace=True)

In [45]:
main_data_df.head()

,전월세매매구분,전용면적(㎡),계약일,금액(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,도로명주소,X좌표,Y좌표,주택가격지수
0,매매,77.75,8,57000.0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,79.100
1,매매,77.75,16,57000.0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,79.213
2,매매,77.75,29,55000.0,7,1988,707.395498,2014,1,10.915088,6.561590,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,79.451
3,매매,77.75,10,59500.0,4,1988,765.273312,2014,11,10.993732,6.640233,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,82.100
4,매매,77.75,28,75000.0,5,1988,964.630225,2016,3,11.225243,6.871745,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06,89.466


In [46]:
main_data_df.shape

(4441027, 17)

In [47]:
%%time
main_data_df.to_csv('./prepped_data/main_data_monthly_converted.csv', index=False)

Wall time: 1min
